In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import mysql.connector
import pandas as pd
import gspread
import json

# Access the environment variables.
mydb_pass = os.getenv('MYDB_PASS')
google_cred = os.getenv('GOOGLE_CRED')
gc = gspread.service_account_from_dict(json.loads(google_cred.replace('"', "'").replace("'", '"')))

In [2]:
# pordreplica.cqof4esbua2o.us-west-2.rds.amazonaws.com
host="production.cqof4esbua2o.us-west-2.rds.amazonaws.com"
user="leadershipDashboard"
password=mydb_pass
database="ck_internal_data"
port=3306

In [3]:
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password,
  database=database,
  port=port
)

mycursor = mydb.cursor()

In [4]:
from sqlalchemy import create_engine

def create_mysql_engine(user, password, host, port, database):
    connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(connection_string)
    return engine


In [5]:
# Get info from google sheet
id_mapping = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NzExg0HSoyrpTSkd9A49EOB4XAWZCh00w9An5p3rKa0/edit#gid=726799529')
ft_logos_gs = pd.DataFrame(id_mapping.worksheet('ft_logos').get_all_records())
ft_logos_gs.head()

,corporate_id,hubspot_id,record_type,corporate_client
0,91437895,4104406488,Managing Office,Aveda Fredric's Institute - Corporate Office
1,91985643,7529688760,Franchise,Aveda Fredric's Institute - Wilmington
2,91460906,4108671409,Managing Office,Douglas J Aveda Institute - Corporate Office
3,91460907,7722083416,Franchise,Douglas J Aveda Institute - Grand Rapids
4,91460911,7722270265,Franchise,Douglas J Aveda Institute - East Lansing


In [6]:
def dataframe_to_mysql(df, table_name, engine, if_exists="fail", index=False):
    """
    Load a DataFrame into a MySQL table.
    
    Parameters:
    - df (pandas.DataFrame): DataFrame to load.
    - table_name (str): Name of the table in the MySQL database.
    - engine (sqlalchemy.engine): SQLAlchemy engine instance.
    - if_exists (str): What to do if the table already exists. Options: "fail", "replace", "append". Default: "fail".
    - index (bool): Whether to write the DataFrame's index to the table. Default: False.
    """
    df.to_sql(table_name, engine, if_exists=if_exists, index=index)

In [7]:
# Write data to MySQL
engine = create_mysql_engine(user, password, host, 3306, database)
dataframe_to_mysql(ft_logos_gs, 'ft_logos', engine, if_exists="replace")

In [9]:
# query = "DROP TABLE IF EXISTS ft_logos;"
# mycursor.execute(query)

In [8]:
query = """ALTER TABLE ft_logos
ADD PRIMARY KEY (corporate_id);
"""
mycursor.execute(query)

In [11]:
query = """
ALTER TABLE ft_logos
MODIFY corporate_id VARCHAR(255);
"""
mycursor.execute(query)

In [ ]:
query = """
ALTER TABLE ft_logos
MODIFY hubspot_id VARCHAR(255);
"""
mycursor.execute(query)

In [1]:
query = '''ALTER TABLE ck_internal_data.ft_sites 
            ADD CONSTRAINT ft_sites_FK FOREIGN KEY (corporate_id) 
                REFERENCES ck_internal_data.ft_logos(corporate_id);
'''
mycursor.execute(query)